# Sign +Narrative

In [7]:
import numpy as np
from scipy.io import loadmat
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import h5py

In [8]:
import numpy as np
import h5py
from scipy.io import loadmat

def load_svar_output(filepath):
    try:
        # Try legacy (MATLAB v5) reader
        mat = loadmat(filepath, squeeze_me=True)
        svar_output_struct = mat['svar_output']
        return {
            name: np.array(svar_output_struct[name]) 
            for name in svar_output_struct.dtype.names
        }
    except NotImplementedError:
        # If file is v7.3 (HDF5), use h5py
        with h5py.File(filepath, 'r') as f:
            return {k: f[k][()] for k in f.keys()}


# === Normalize IRFs to a target shock size (in bps)
def normalize_irfs_to_shock_size_median(irfs, varnames, shock_var='Fed Funds Rate', target_shock_size=0.25):
    idx_shock = varnames.index(shock_var)
    init_median = np.median(irfs[idx_shock, 0, :])
    scale = target_shock_size / init_median if init_median != 0 else 1.0
    return irfs * scale * 100

def clean_irfs(irfs_raw):
    """
    Convert a nested IRF object (from Julia or MATLAB .mat) into a clean float array of shape (nvar, horizon, draws).
    """
    # Step 1: If this is a 1-element object array (common in Julia exports), unwrap it
    if isinstance(irfs_raw, np.ndarray) and irfs_raw.dtype == np.object_:
        if irfs_raw.size == 1:
            irfs_raw = irfs_raw.item()

    # Step 2: Try to convert to a float array (this will fail if it's ragged or inconsistent)
    irfs = np.array(irfs_raw, dtype=float)

    # Step 3: Drop draws with NaNs in any variable or horizon
    valid_draws = ~np.isnan(irfs).any(axis=(0, 1))

    return irfs[:, :, valid_draws]


# === IRF comparison plot ===
def plot_comparison_2(irfs1, irfs2, label1, label2, color1, color2, varnames, horizons):
    nvar = irfs1.shape[0]
    ncols = 3
    nrows = int(np.ceil(nvar / ncols))
    fig = make_subplots(rows=nrows, cols=ncols, subplot_titles=[f"<b>{v}</b>" for v in varnames[:nvar]])

    for var in range(nvar):
        row = var // ncols + 1
        col = var % ncols + 1

        for draws, label, color in zip([irfs1, irfs2], [label1, label2], [color1, color2]):
            median = np.median(draws[var, :, :], axis=1)
            lower = np.percentile(draws[var, :, :], 16, axis=1)
            upper = np.percentile(draws[var, :, :], 84, axis=1)

            legend_label = {
                'RR': 'Uhlig (2005)',
                'Carriero': 'Blended',
                'Dawis': 'SVAR-SVSR',
                'Dawis_penalty': 'SVAR-SVSR with penalty',
                'Carriero_nomh': 'Blended without M-H steps'
            }.get(label, f'{label}')

            fig.add_trace(go.Scatter(
                x=horizons, y=median,
                mode='lines',
                line=dict(color=color, width=2),
                showlegend=False
            ), row=row, col=col)

            fig.add_trace(go.Scatter(
                x=np.concatenate([horizons, horizons[::-1]]),
                y=np.concatenate([upper, lower[::-1]]),
                fill='toself',
                fillcolor=color.replace('rgb', 'rgba').replace(')', ',0.5)'),
                line=dict(color='rgba(255,255,255,0)'),
                hoverinfo="skip",
                name=legend_label,
                showlegend=(var == 0)
            ), row=row, col=col)

        fig.add_trace(go.Scatter(
            x=horizons, y=np.zeros_like(horizons),
            mode='lines',
            line=dict(color='black', dash='dash'),
            showlegend=False
        ), row=row, col=col)

    fig.update_layout(
        height=500,
        width=1000,
        template='plotly_white',
        font=dict(size=20),
        legend=dict(
            font=dict(size=20, color='black'),
            orientation='h',
            yanchor='bottom',
            y=1.1,
            xanchor='center',
            x=0.5
        ),
        margin=dict(t=80)
    )

    fig.update_xaxes(tickfont=dict(size=16, color='black'))
    fig.update_yaxes(title_font=dict(size=20, color='black'), tickfont=dict(size=14, color='black'))
    fig.for_each_annotation(lambda a: a.update(font=dict(size=16, color='black')))
    fig.show()


In [9]:
varnames = ['GDP', 'GDP Deflator', 'Commodity Price Index', 'Total Reserve', 'Non-borrowed Reserves', 'Fed Funds Rate']

svar_narrative_rubioramirez = load_svar_output('output/irfs_narrative_rubioramirez.mat')
svar_narrative_dawis_regime = load_svar_output('output/irfs_narrative_dawis_regime.mat')
svar_narrative_dawis_regime_penalty = load_svar_output('output/irfs_narrative_dawis_regime_penalty.mat')
svar_narrative_carriero_regime = load_svar_output('output/irfs_narrative_carriero_regime.mat')

irfs_narrative_rubioramirez = clean_irfs(svar_narrative_rubioramirez['sign_irfs_temp'])
irfs_narrative_dawis_regime = clean_irfs(svar_narrative_dawis_regime['sign_irfs_temp'])
irfs_narrative_dawis_regime_penalty = clean_irfs(svar_narrative_dawis_regime_penalty['sign_irfs_temp'])
irfs_narrative_carriero_regime = clean_irfs(svar_narrative_carriero_regime['sign_irfs_temp'])

In [10]:
# Define horizons properly
horizons = np.arange(60)

plot_comparison_2(
    irfs_narrative_rubioramirez, irfs_narrative_dawis_regime,
    label1='AD-RR', label2='SVAR-SVSR',
    color1='rgb(64,64,64)', color2='rgb(255,0,0)',
    varnames=varnames, horizons=horizons
)

plot_comparison_2(
    irfs_narrative_dawis_regime, irfs_narrative_carriero_regime,
    label1='SVAR-SVSR', label2='No Variance Rotation',
    color1='rgb(255,0,0)', color2='rgb(33,150,243)',
    varnames=varnames, horizons=horizons
)

plot_comparison_2(
    irfs_narrative_dawis_regime, irfs_narrative_dawis_regime_penalty,
    label1='SVAR-SVSR', label2='SVAR-SVSR with penalty',
    color1='rgb(255,0,0)', color2='rgb(0,114,178)',
    varnames=varnames, horizons=horizons
)

# Sign + Elasticity

In [11]:
def plot_irfs_from_4d_levels_two_models(
    irfs_a,
    irfs_b,
    labels=("Model A", "Model B"),
    colors=("rgb(0,0,0)", "rgb(255,0,0)"),
    varnames=None,
    horizons=None,
    shock_labels=None,
    normalize_to=(-1.0, 1.0, 1.0),
    cumsum_vars=(0,),
    ci=(16, 84),
):
    """
    Accepts 3D (nvar, horizon, nshock*ndraws or nshock) or 4D (nvar, horizon, nshock, ndraws).
    If 3D with last dim == nshock*ndraws, convert externally before calling.
    """
    import numpy as np
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots

    def _rgba(alpha_src, a=0.2):
        # "rgb(r,g,b)" -> "rgba(r,g,b,alpha)"
        return alpha_src.replace('rgb', 'rgba').replace(')', f',{a})')

    def _to4d(x):
        x = np.asarray(x)
        if x.ndim == 4:
            return x
        if x.ndim == 3:
            # Treat as (nvar, H, nshock) -> single draw
            return x[..., np.newaxis]
        raise ValueError(f"IRFs must be 3D or 4D, got shape {x.shape}")

    def _normalize_and_cumsum(irfs, normalize_to, cumsum_vars):
        nvar, H, nshock, ndraws = irfs.shape
        out = irfs.copy()
        norm = np.asarray(normalize_to, float)
        if norm.size < nshock:
            norm = np.pad(norm, (0, nshock - norm.size), constant_values=1.0)
        for s in range(nshock):
            own = irfs[s, 0, s, :]  # (ndraws,)
            # Avoid divide-by-zero; if own==0, leave scale=1
            scale = np.where(own != 0, norm[s] / own, 1.0)
            # out[:, :, s, :] = irfs[:, :, s, :] * scale[np.newaxis, np.newaxis, :]
            for v in cumsum_vars:
                if 0 <= v < nvar:
                    out[v, :, s, :] = np.cumsum(out[v, :, s, :], axis=0)
        return out

    # --- Ensure 4D ---
    irfs_a = _to4d(irfs_a)
    irfs_b = _to4d(irfs_b)

    # --- Basic checks + labels ---
    nvar, H, nshock, _ = irfs_a.shape
    if irfs_b.shape[:3] != (nvar, H, nshock):
        raise ValueError(f"IRF dims must match. A:{irfs_a.shape} vs B:{irfs_b.shape}")

    if varnames is None:
        varnames = [f"Var {i+1}" for i in range(nvar)]
    if horizons is None:
        horizons = np.arange(H)
    else:
        horizons = np.asarray(horizons)
        if len(horizons) != H:
            raise ValueError(f"len(horizons)={len(horizons)} must equal horizon={H}")
    if shock_labels is None:
        shock_labels = [f"Shock {s+1}" for s in range(nshock)]

    # Normalize & cumsum
    irfs_a_scaled = _normalize_and_cumsum(irfs_a, normalize_to, cumsum_vars)
    irfs_b_scaled = _normalize_and_cumsum(irfs_b, normalize_to, cumsum_vars)

    # Subplots: rows = shocks, cols = variables
    fig = make_subplots(
        rows=nshock,
        cols=nvar,
        subplot_titles=[f"<b>{var}</b><br>{shock_labels[s]}" for s in range(nshock) for var in varnames]
    )

    band_label = f"{ci[1]-ci[0]}"

    for j in range(nshock):       # rows: shocks
        for i in range(nvar):     # cols: variables
            row, col = j + 1, i + 1

            # --- Model A ---
            data_a = irfs_a_scaled[i, :, j, :]
            med_a = np.median(data_a, axis=1)
            lo_a  = np.percentile(data_a, ci[0], axis=1)
            hi_a  = np.percentile(data_a, ci[1], axis=1)

            fig.add_trace(go.Scatter(
                x=np.concatenate([horizons, horizons[::-1]]),
                y=np.concatenate([hi_a, lo_a[::-1]]),
                fill='toself',
                fillcolor=_rgba(colors[0], 0.5),
                line=dict(color='rgba(255,255,255,0)'),
                name=f"{labels[0]}",
                hoverinfo='skip',
                showlegend=(row == 1 and col == 1)
            ), row=row, col=col)

            fig.add_trace(go.Scatter(
                x=horizons, y=med_a, mode='lines',
                line=dict(color=colors[0], width=2),
                name=labels[0],
                showlegend=False
            ), row=row, col=col)

            # --- Model B ---
            data_b = irfs_b_scaled[i, :, j, :]
            med_b = np.median(data_b, axis=1)
            lo_b  = np.percentile(data_b, ci[0], axis=1)
            hi_b  = np.percentile(data_b, ci[1], axis=1)

            fig.add_trace(go.Scatter(
                x=np.concatenate([horizons, horizons[::-1]]),
                y=np.concatenate([hi_b, lo_b[::-1]]),
                fill='toself',
                fillcolor=_rgba(colors[1], 0.5),
                line=dict(color='rgba(255,255,255,0)'),
                name=f"{labels[1]}",
                hoverinfo='skip',
                showlegend=(row == 1 and col == 1)
            ), row=row, col=col)

            fig.add_trace(go.Scatter(
                x=horizons, y=med_b, mode='lines',
                line=dict(color=colors[1], width=2),
                name=labels[1],
                showlegend=False
            ), row=row, col=col)

            # Zero line
            fig.add_trace(go.Scatter(
                x=horizons, y=np.zeros_like(horizons), mode='lines',
                line=dict(color='black', dash='dash'),
                showlegend=False
            ), row=row, col=col)

    fig.update_layout(
        height=300 * nshock,
        width=400 * nvar,
        template='plotly_white',
        font=dict(size=20),
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=1.1,
            xanchor='center',
            x=0.5,
            font=dict(size=16, color='black')   # ✅ move color inside font
        ),
        margin=dict(t=70)
    )

    fig.update_xaxes(tickfont=dict(size=14, color = 'black'))
    fig.update_yaxes(tickfont=dict(size=14, color = 'black'))
    fig.for_each_annotation(lambda a: a.update(font=dict(size=14, color = 'black')))
    fig.show()


In [12]:
# Load all outputs
svar_uhlig = load_svar_output('output/killian/irfs_uhlig.mat')
svar_dawis = load_svar_output('output/killian/irfs_dawis_regime.mat')
svar_dawis_penalty = load_svar_output('output/killian/irfs_dawis_regime_penalty.mat')
svar_carriero = load_svar_output('output/killian/irfs_carriero_regime.mat')

# Extract and clean
irfs_uhlig = clean_irfs(svar_uhlig['sign_irfs_temp'])
irfs_dawis = clean_irfs(svar_dawis['sign_irfs_temp'])
irfs_dawis_penalty = clean_irfs(svar_dawis_penalty['sign_irfs_temp'])
irfs_carriero = clean_irfs(svar_carriero['sign_irfs_temp'])

# Convert stacked-by-shock 3D IRFs -> 4D (nvar, horizon, nshock, ndraws)
def to4d_stacked_by_shock(irfs_3d, nshock):
    import numpy as np
    irfs_3d = np.asarray(irfs_3d)
    nvar, H, N = irfs_3d.shape
    if N % nshock != 0:
        raise ValueError(f"Last dim {N} not divisible by nshock={nshock}")
    ndraws = N // nshock
    # Assumes blocks like [shock1 draws..., shock2..., shock3...]
    return irfs_3d.reshape(nvar, H, nshock, ndraws)

nshock = 3
irfs_uhlig    = to4d_stacked_by_shock(irfs_uhlig, nshock)      # -> (3, 17, 3, 1000)
irfs_dawis    = to4d_stacked_by_shock(irfs_dawis, nshock)      # -> (3, 17, 3, 1000)
irfs_dawis_penalty    = to4d_stacked_by_shock(irfs_dawis_penalty, nshock)      # -> (3, 17, 3, 1000)
irfs_carriero = to4d_stacked_by_shock(irfs_carriero, nshock)   # -> (3, 17, 3, 1000)

In [13]:
plot_irfs_from_4d_levels_two_models(
    irfs_uhlig, irfs_dawis,
    labels=("KM12", "SVAR-SVSR"),
    colors=("rgb(64,64,64)", "rgb(255,0,0)"),
    varnames=['Oil Production Growth', 'Economic Activity Index', 'Real Oil Price'],
    horizons=np.arange(17),
    shock_labels=['Oil supply shock', 'Aggregate demand shock', 'Oil-specific demand shock'],
    normalize_to=[-1.0, 1.0, 1.0],  # normalize to -1, 1, 1 for each shock
    cumsum_vars=(0,),   # cumulative sum variable(s)
    ci=(16, 84)
)

plot_irfs_from_4d_levels_two_models(
    irfs_dawis, irfs_carriero,
    labels=("SVAR-SVSR", "No Variance Rotation"),
    colors=("rgb(255,0,0)", "rgb(33,150,243)"),  # red and lighter blue
    varnames=['Oil Production Growth', 'Economic Activity Index', 'Real Oil Price'],
    horizons=np.arange(17),
    shock_labels=['Oil supply shock', 'Aggregate demand shock', 'Oil-specific demand shock'],
    normalize_to=[-1.0, 1.0, 1.0],  # normalize to -1, 1, 1 for each shock
    cumsum_vars=(0,),   # cumulative sum variable(s)
    ci=(16, 84)
)

plot_irfs_from_4d_levels_two_models(
    irfs_dawis, irfs_dawis_penalty,
    labels=("SVAR-SVSR", "SVAR-SVSR-Penalty"),
    colors=("rgb(255,0,0)", "rgb(0,114,178)"),
    varnames=['Oil Production Growth', 'Economic Activity Index', 'Real Oil Price'],
    horizons=np.arange(17),
    shock_labels=['Oil supply shock', 'Aggregate demand shock', 'Oil-specific demand shock'],
    normalize_to=[-1.0, 1.0, 1.0],  # normalize to -1, 1, 1 for each shock
    cumsum_vars=(0,),   # cumulative sum variable(s)
    ci=(16, 84)
)

# Sign + Zero

In [14]:
varnames = [
    'Adjusted TFP','Stock Prices','Consumption', 'Real Interest Rate','Hours Worked'
]

# Load all outputs
svar_uhlig = load_svar_output('output/SignZero/irfs_uhlig.mat')
svar_dawis = load_svar_output('output/SignZero/irfs_dawis_regime.mat')
svar_dawis_penalty = load_svar_output('output/SignZero/irfs_dawis_regime_penalty.mat')
svar_carriero = load_svar_output('output/SignZero/irfs_carriero_regime.mat')

# Extract and clean
irfs_uhlig = clean_irfs(svar_uhlig['sign_irfs_temp'])
irfs_dawis = clean_irfs(svar_dawis['sign_irfs_temp'])
irfs_dawis_regime1 = svar_dawis['sign_irf_regime'].item()[:,:,:,0]
irfs_dawis_penalty = clean_irfs(svar_dawis_penalty['sign_irfs_temp'])
irfs_carriero = clean_irfs(svar_carriero['sign_irfs_temp'])

In [15]:
import numpy as np

# === Normalize IRFs to a target impact for EVERY draw
# irfs: array (nvar, horizon, ndraws) for a single shock
# shock_var: variable name used for normalization (impact at h=0)
# target_shock_size: target impact in "unit" units (see unit)
# unit: 'raw' (no conversion), 'percent' (×100), or 'bps' (×10_000)
# use_abs: if True, match the absolute size (keep each draw's sign)
def normalize_irfs_to_shock_size_all_draws(
    irfs, varnames, shock_var='Stock Prices', target_shock_size=0.035,
    unit='raw', use_abs=False
):
    irfs = np.asarray(irfs, dtype=float)
    idx = varnames.index(shock_var)

    # per-draw initial impact on the normalization variable
    impact0 = irfs[idx, 0, :]           # shape: (ndraws,)

    if use_abs:
        denom = np.where(impact0 != 0.0, np.abs(impact0), 1.0)
        scale = target_shock_size / denom
        # restore original sign
        scale *= np.sign(impact0 + (impact0 == 0.0))
    else:
        denom = np.where(impact0 != 0.0, impact0, 1.0)
        scale = target_shock_size / denom

    # broadcast over (nvar, horizon, ndraws)
    irfs_scaled = irfs * scale[np.newaxis, np.newaxis, :]

    if unit == 'percent':
        irfs_scaled *= 100.0          # levels in percentage points
    elif unit == 'bps':
        irfs_scaled *= 10_000.0       # levels in basis points

    return irfs_scaled


In [16]:
horizons = np.arange(irfs_uhlig.shape[1])  # or your own array

plot_comparison_2(
    irfs_uhlig, irfs_dawis,
    label1='ARW', label2='SVAR-SVSR',
    color1='rgb(64,64,64)', color2='rgb(255,0,0)',
    varnames=varnames, horizons=horizons
)

plot_comparison_2(
    irfs_dawis, irfs_carriero,
    label1='SVAR-SVSR', label2='No Variance Rotation',
    color1='rgb(255,0,0)', color2='rgb(33,150,243)',
    varnames=varnames, horizons=horizons
)

plot_comparison_2(
    irfs_dawis, irfs_dawis_penalty,
    label1='SVAR-SVSR', label2='SVAR-SVSR-Penalty',
    color1='rgb(255,0,0)', color2='rgb(0,114,178)',
    varnames=varnames, horizons=horizons
)